In [46]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
from datetime import datetime

In [47]:
link = []
data = []
contents_list = []

#저장할 파일이름 지정
filename = '갤럭시s22'

for page in tqdm(range(int(input()))): #원하는 페이지만큼 입력
#     query = '%BE%C6%C0%CC%C6%D0%B5%E5+%C7%C1%B7%CE+12.9+5%BC%BC%B4%EB' # << 아이패드 프로 12.9 5세대의 쿼리 다른 검색어는 별도 변환 필요
#     query = '%B0%B6%B7%B0%BD%C3+%C5%C7+s6+lite' # << 갤럭시 탭 s6 lite 쿼리
    query = '%B0%B6%B7%B0%BD%C3+s22'  #<<갤럭시 s22 쿼리

    menuid = 339
    # 메뉴아이디 749 = 테블릿, 339 = 휴대폰
    
    url = f"https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=10050146&search.menuid={menuid}&search.media=0&search.searchdate=all&search.exact=&search.include=&userDisplay=50&search.exclude=&search.onSale=1&search.option=3&search.sortBy=date&search.searchBy=1&search.searchBlockYn=0&search.includeAll=&search.query={query}&search.viewtype=title&search.page={1+page}"

    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    
#     contents = soup.select('.inner_list .article') #단순 페이지 링크 구하는 태그
    
    id = soup.select('.inner_number') #판매글의 아이디 구하기
    
    for i in id: #판매글의 아이디를 통해 판매 정보을 포함한 jason 링크 리스트 만들기
        contents_list.append(f'https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10050146/articles/{i.text}?query=&menuId={menuid}&useCafeId=true&requestFrom=A')

    

50


100%|██████████| 50/50 [00:44<00:00,  1.12it/s]


In [48]:
junggo = []
# json 파일에서 제목, 가격, 판매자, 설명 추출해 리스트로 저장
for url in tqdm(contents_list):
    response = requests.get(url)
    data = response.json()
    try:
        title = data["result"]["article"]['subject']
        price = data["result"]["saleInfo"]['price']
        seller = data["result"]["article"]['writer']['nick']
        desc = BeautifulSoup(data["result"]["article"]["contentHtml"], 'html.parser').text
        #타임스탬프
        date = datetime.fromtimestamp(data['result']['article']['writeDate']/1000).strftime('%y-%m-%d')
        
        junggo.append({'title' : title, 'price' : price, 'date' : date,'seller' : seller, 'desc' : desc})
    
    except:
        print('error')
        continue
        #크롤링 도중 실시간으로 게시글이 삭제되는 문제 발견
        #예외처리 시도
    

  3%|▎         | 65/2348 [00:20<10:40,  3.56it/s]

error


  3%|▎         | 80/2348 [00:25<11:42,  3.23it/s]

error


 26%|██▌       | 612/2348 [03:20<09:18,  3.11it/s]

error


 70%|███████   | 1648/2348 [08:59<03:22,  3.45it/s]

error


100%|██████████| 2348/2348 [12:33<00:00,  3.12it/s]


In [49]:
#데이터프레임화
junggo = pd.DataFrame(junggo)
junggo

,title,price,date,seller,desc
0,"[매입]갤럭시Z폴드3 S22울트라(자급제폰,박스폰,당일개통폰.중고폰",1990000,22-07-06,24시간거래가능,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​24시방문...
1,갤럭시 S22 울트라 1tb 미개봉 스카이블루,1450000,22-07-06,에눌사절이요,\n\n\n\n\n\n\n\n\n👆 중고나라에서 5천원 상품권 드리는 거 아시나요?...
2,갤럭시s22울트라 256gb 그린판매합니다,930000,22-07-06,24시간판매중,\n\n\n\n\n\n\n\n\n👆 중고나라에서 스타벅스&amp;GS25 드리는 거...
3,갤럭시 s22플러스,700000,22-07-06,레몬스텔라,\n\n\n\n\n\n\n\n\n👆 중고나라에서 5천원 상품권 드리는 거 아시나요?...
4,"[매입] 갤럭시Z폴드3 S22 울트라,박스폰,자급제미개봉,당일개통삽니다",99999999,22-07-06,24시간폰매입합니다,"\n\n\n\n\n\n\n\n\n👆 위 중고나라 앱 다운로드 링크를 통해, 앱 다운..."
...,...,...,...,...,...
2339,"미개봉, 갤럭시 S22 , 팬텀화이트 , 256G , 필름,투명젤리케이스함께, 제조...",740000,22-04-21,당근이지당근,"\n\n\n\n\n\n\n\n\n미개봉, 갤럭시 S22 , 팬텀화이트 , 256G ..."
2340,갤럭시s22 울트라 플러스/제트폴드3/제트플립3 삼성 전기종 매입!,1980000,22-04-20,일산매입업체1,"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​​"" 휴대..."
2341,갤럭시S22플러스 256G 화이트/블랙/그린색상 단순개봉 미사용 자급제 단말기 판매...,880000,22-04-20,폰사요들2,\n\n\n\n\n\n\n\n\n갤럭시S22플러스 256G 화이트/블랙/그린색상 미...
2342,갤럭시S22플러스 256G 화이트/블랙/그린색상 단순개봉 미사용 자급제 단말기 판매...,880000,22-04-20,폰사요들,\n\n\n\n\n\n\n\n\n갤럭시S22플러스 256G 화이트/블랙/그린색상 미...


In [50]:
#설명 전처리
junggo.reset_index(drop = True)
junggo['desc'] = junggo['desc'] \
.replace(r'[^가-힣 ]', ' ', regex=True) \
.replace("'", '') \
.replace(r'\s+', ' ', regex=True) \
.str.strip() \
.str[:255] \
.replace({'위 중고나라 앱 다운로드 링크를 통해 앱 다운로드하여 판매 및 구매 게시글을 등록해 주세요':'', '중고나라에서 천원 상품권 드리는 거 아시나요 더보기 버튼을 눌러 확인해 주세요':'', '중고나라에서 스타벅스 드리는 거 아시나요 더보기 버튼을 눌러 확인해 주세요' : ''}, regex=True)
junggo = junggo[junggo['desc'].str.strip().astype(bool)]

In [51]:
junggo = junggo.drop_duplicates(['desc'], keep = 'first', ignore_index = True)

In [52]:
#매입 관련 데이터 단순삭제

junggo = junggo[~junggo['seller'].str.contains('매입|삽니다|최고가')]
junggo = junggo[~junggo['title'].str.contains('매입|삽니다|최고가')]
junggo = junggo[~junggo['desc'].str.contains('매입|삽니다|최고가')]

In [53]:
junggo.reset_index(drop = True, inplace = True)

In [54]:
junggo

,title,price,date,seller,desc
0,갤럭시 S22 울트라 1tb 미개봉 스카이블루,1450000,22-07-06,에눌사절이요,미개봉 자급제라 설명할게 없네요직거래 택배 가능에눌 바로 오실분만 연락주세요
1,갤럭시 s22플러스,700000,22-07-06,레몬스텔라,올해 월에 샀어요아이폰 쓰다가 삼성페이 쓰고싶어서 샀는데 아이폰이 더 편해서 팝니...
2,부산 갤럭시S22울트라 화이트 특S급 리퍼폰 센터올갈이 초꿀매,950000,22-07-06,부산중고폰1등폰고래,신뢰를 우선으로 거래하는 중고나라 파트너 제휴사 부산 중고폰 등 매장 폰고래입니다 ...
3,갤럭시 S22 청녹색 판매 (개인),790000,22-07-06,도다파달,월 말에 샀는데 저랑 안맞아서 처분합니다 서울 강동구 둘둘사공 오삼육칠 월말에 이틀...
4,갤럭시 S22 화이트 256기가 SSS급 기기 팝니다,640000,22-07-06,꺄하핫82,사진과 동일한 제품 최초통화일 최초통신사 기스 없고 잔상 없는 급 기기 단품 사유심...
...,...,...,...,...,...
316,갤럭시 s22 플러스 풀박스 미개봉 새제품,880000,22-04-22,adaeng_s2,새제품 미개통 미개봉
317,갤럭시 s22울트라 256gb 다크그린 사용한적없는 새상품 판매합니다,1000000,22-04-22,hjwon0613,머칠전 선물받았는데 손이 작은 저로서는 넘 크고 무겁게 느껴져서 글 올려요어느통신사...
318,갤럭시 s22 울트라 256gb 다크그린 새정품 팝니다.,1000000,22-04-22,hjwon0613,며칠전 선물받았는데 제손이 작아서 핸드폰이 더 크게 느껴지고 무겁게 까지 느껴지고...
319,갤럭시 S22울트라 512 자급제폰 팝니다,1050000,22-04-22,큰물,기스없는 깨끗한 상태입니다직거래는 용인 기흥구 가능합니다


In [55]:
junggo.to_csv(f"{filename}_크롤링.csv" ,encoding="utf-8", index_label=False)
junggo.head()

,title,price,date,seller,desc
0,갤럭시 S22 울트라 1tb 미개봉 스카이블루,1450000,22-07-06,에눌사절이요,미개봉 자급제라 설명할게 없네요직거래 택배 가능에눌 바로 오실분만 연락주세요
1,갤럭시 s22플러스,700000,22-07-06,레몬스텔라,올해 월에 샀어요아이폰 쓰다가 삼성페이 쓰고싶어서 샀는데 아이폰이 더 편해서 팝니...
2,부산 갤럭시S22울트라 화이트 특S급 리퍼폰 센터올갈이 초꿀매,950000,22-07-06,부산중고폰1등폰고래,신뢰를 우선으로 거래하는 중고나라 파트너 제휴사 부산 중고폰 등 매장 폰고래입니다 ...
3,갤럭시 S22 청녹색 판매 (개인),790000,22-07-06,도다파달,월 말에 샀는데 저랑 안맞아서 처분합니다 서울 강동구 둘둘사공 오삼육칠 월말에 이틀...
4,갤럭시 S22 화이트 256기가 SSS급 기기 팝니다,640000,22-07-06,꺄하핫82,사진과 동일한 제품 최초통화일 최초통신사 기스 없고 잔상 없는 급 기기 단품 사유심...
